In [ ]:
import cv2
import matplotlib.pyplot as plt

# Path to the image
image_path = "/kaggle/input/tumor-injection-attack/ATTACKED/sub-BrainAge000034_T1w_axial_center.png"

# Read the image (OpenCV reads images in BGR format)
image = cv2.imread(image_path)

# Convert the image from BGR to RGB format
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Display the image
plt.figure(figsize=(8, 8))
plt.imshow(image_rgb)
plt.axis('on')
plt.show()


In [ ]:
import cv2
import numpy as np
import os
import shutil

# Function to classify a single image
def classify_brain_image(image_path, box_size=5, white_threshold=0.3):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image: {image_path}")
        return None, None
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    height, width = gray.shape
    tumor_detected = False
    for y in range(0, height - box_size + 1, box_size):
        for x in range(0, width - box_size + 1, box_size):
            box = gray[y:y+box_size, x:x+box_size]
            total_pixels = box_size * box_size
            white_pixels = np.sum(box == 255)
            white_ratio = white_pixels / total_pixels
            
            if white_ratio >= white_threshold:
                tumor_detected = True
                break
        if tumor_detected:
            break
    
    return image, "Tumor" if tumor_detected else "Non-Tumor"

# Input and output directories
input_folder = "/kaggle/input/tumor-injection-attack/ATTACKED"
output_tumor_folder = "/kaggle/working/Tumor"
output_non_tumor_folder = "/kaggle/working/Non-Tumor"

# Create output folders if they don't exist
os.makedirs(output_tumor_folder, exist_ok=True)
os.makedirs(output_non_tumor_folder, exist_ok=True)

# Process all images in the input folder
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(input_folder, filename)
        image, classification = classify_brain_image(image_path)
        
        if image is not None:
            if classification == "Tumor":
                cv2.imwrite(os.path.join(output_tumor_folder, filename), image)
            else:
                cv2.imwrite(os.path.join(output_non_tumor_folder, filename), image)

print("Classification complete. Saved images to Tumor and Non-Tumor folders.")


In [ ]:

# Paths to the folders where classified images are stored
tumor_folder = "/kaggle/working/Tumor"
non_tumor_folder = "/kaggle/working/Non-Tumor"

def show_images_from_folder(folder_path, title, max_images=10):
    images = []
    filenames = os.listdir(folder_path)
    filenames = [f for f in filenames if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    filenames = filenames[:max_images]
    
    for filename in filenames:
        image_path = os.path.join(folder_path, filename)
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        images.append(image_rgb)
    
    # Display images in a grid
    plt.figure(figsize=(15, 5))
    for i, img in enumerate(images):
        plt.subplot(2, 5, i+1)
        plt.imshow(img)
        plt.axis('off')
    plt.suptitle(title, fontsize=16)
    plt.show()

# Show first 10 Non-Tumor images
show_images_from_folder(non_tumor_folder, "First 10 Non-Tumor Images")

# Show first 10 Tumor images
show_images_from_folder(tumor_folder, "First 10 Tumor Images")
